## Load dataset

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_data = pd.read_csv('google-landmarks-dataset/train.csv')
test_data = pd.read_csv('google-landmarks-dataset/test.csv')

In [ ]:
train_data.head(5)

## Sample the data

In [ ]:
landmark_list = [str(x) for x in range(1000, 3001)]
train_data_sample = train_data[train_data['landmark_id'].isin(landmark_list)]

In [ ]:
print('sample train data:', len(train_data_sample))

In [ ]:
colors = np.array(['#4285f4','#34a853','#fbbc05','#ea4335'])
#Define the order in which to display the graph
order = ['1-5','5-10','10-50','50-100','100-200','200-500','>=500']
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,5))

def plot_distribution(data_f, data_k, axis):
    x = data_f.landmark_id.value_counts().index
    y = pd.DataFrame(data_f.landmark_id.value_counts())
    
    y['Number of images'] = np.where(y.landmark_id >= 500, '>=500', y['landmark_id'])
    y['Number of images'] = np.where((y.landmark_id >= 200) & (y.landmark_id < 500), '200-500', y['Number of images'])
    y['Number of images'] = np.where((y.landmark_id >= 100) & (y.landmark_id < 200), '100-200', y['Number of images'])
    y['Number of images'] = np.where((y.landmark_id >= 50) & (y.landmark_id < 100), '50-100', y['Number of images'])
    y['Number of images'] = np.where((y.landmark_id >= 10) & (y.landmark_id < 50), '10-50', y['Number of images'])
    y['Number of images'] = np.where((y.landmark_id >= 5) & (y.landmark_id < 10), '5-10', y['Number of images'])
    y['Number of images'] = np.where((y.landmark_id >= 1) & (y.landmark_id < 5), '1-5', y['Number of images'])

    y['Number of images'].value_counts().loc[order].plot(kind = 'bar',color = colors,width = 0.8, ax=axis)
    axis.set_xlabel('Number of images')
    axis.set_ylabel('Number of classes')
    axis.set_title(data_k)

plot_distribution(train_data, 'Original', ax1)
plot_distribution(train_data_sample, 'Sample', ax2)

In [ ]:
url_df = train_data_sample[train_data_sample.url.apply(lambda x: len(x.split('/')) > 1)]

In [ ]:
url_test = url_df.url.apply(lambda x: x.split('/'))

In [ ]:
url_test_tail = url_test.apply(lambda x: x[-2])